In [10]:
from ultralytics import YOLO
import cv2
import os
import numpy as np

In [4]:
model = YOLO("yolov8s.pt")

output_path = "out_detect.mp4"

In [5]:
cap = cv2.VideoCapture("shop_lifter_0.mp4")
if not cap.isOpened():
    print("cannot open the video")

In [7]:
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
def crop_person_from_video(
    model,
    input_path, 
    output_path, 
    expand_ratio=0.3, 
    output_size=(256, 256)
):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(
        output_path, 
        cv2.VideoWriter_fourcc(*'mp4v'), 
        fps, 
        output_size
    )

    boxes = []
    frames = []

    # Step 1: detect all boxes
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        results = model(frame, verbose=False)
        detections = results[0].boxes.xyxy.cpu().numpy() if len(results[0].boxes) > 0 else []
        if len(detections) > 0:
            # take biggest box (person)
            areas = (detections[:,2]-detections[:,0])*(detections[:,3]-detections[:,1])
            box = detections[np.argmax(areas)]
            boxes.append(box)
        else:
            boxes.append(None)
    cap.release()

    # Step 2: find valid range (first and last frame where a person exists)
    valid_indices = [i for i,b in enumerate(boxes) if b is not None]
    if not valid_indices:
        print(f"No detections found in {input_path}")
        return
    
    first_valid, last_valid = valid_indices[0], valid_indices[-1]

    # Step 3: fill missing boxes in the middle only
    for i in range(first_valid+1, last_valid):
        if boxes[i] is None:
            # interpolate between previous and next available boxes
            prev_idx = max(j for j in range(i-1, first_valid-1, -1) if boxes[j] is not None)
            next_idx = min(j for j in range(i+1, last_valid+1) if boxes[j] is not None)
            if prev_idx is not None and next_idx is not None:
                boxes[i] = (boxes[prev_idx] + boxes[next_idx]) / 2

    # Step 4: crop & write
    for i, frame in enumerate(frames):
        if i < first_valid or i > last_valid:
            crop = np.zeros((*output_size, 3), dtype=np.uint8)  # black frame
        else:
            box = boxes[i]
            if box is not None:
                x1, y1, x2, y2 = box
                w, h = x2 - x1, y2 - y1

                # expand box
                x1 = max(0, int(x1 - expand_ratio * w))
                y1 = max(0, int(y1 - expand_ratio * h))
                x2 = min(frame_width, int(x2 + expand_ratio * w))
                y2 = min(frame_height, int(y2 + expand_ratio * h))

                crop = frame[y1:y2, x1:x2]
                crop = cv2.resize(crop, output_size)
            else:
                crop = np.zeros((*output_size, 3), dtype=np.uint8)
        out.write(crop)
    
    out.release()
    print(f"Saved cropped video to {output_path}")


In [37]:
crop_person_from_video(
    model = model,
    input_path="shop_lifter_110.mp4",
    output_path="cropped_person110_1.mp4",
    output_size=(256, 256),
    expand_ratio = 0.3
)


Saved cropped video to cropped_person110_1.mp4


In [ ]:
# 🧠 Load model (already downloaded)
model = YOLO("yolov8s.pt")

# 📂 Input and output directories
input_root = "Shop DataSet"
output_root = "Processed_Dataset"

# ✅ Ensure output directory exists
os.makedirs(output_root, exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(input_root):
    class_input_dir = os.path.join(input_root, class_name)
    class_output_dir = os.path.join(output_root, class_name)
    os.makedirs(class_output_dir, exist_ok=True)

    print(f"\n🔹 Processing class: {class_name}")

    # Loop through all videos in the class folder
    for file in os.listdir(class_input_dir):
        if not file.lower().endswith(('.mp4', '.avi', '.mov')):
            continue

        input_path = os.path.join(class_input_dir, file)
        output_path = os.path.join(class_output_dir, file)

        try:
            print(f"🎬 Cropping: {file}")
            crop_person_from_video(model, input_path, output_path, target_size=(256, 256), num_frames=16)
        except Exception as e:
            print(f"⚠️ Failed to process {file}: {e}")

print("\n✅ All videos processed and saved in 'Processed_Dataset/'")
